In [1]:
# 코랩에 구글 드라이브 마운트
from google.colab import drive
drive.mount("/content/drive")
import warnings
# warnings.filterwarnings(action = "ignore")

# 데이터 처리 모듈
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
import time
from tqdm import tqdm

# 멀티프로세싱
import multiprocessing as mp
from multiprocessing import Pool

# 척도 변환 모듈(표준화, 정규화, 로버스트 정규화, 원핫인코딩(범주형에서 더미변수로 변환) 등)
from sklearn.preprocessing import *

# 결측값 관측 모듈
import missingno as msno

# 시각화 모듈
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
from matplotlib.ticker import ScalarFormatter

# 배열, 행렬 연산 모듈
import numpy as np

# 데이터 분할 모듈(훈련용 / 검증용 / 시험용)
from sklearn.model_selection import train_test_split

# 통계적 가설검정 및 계량화 모듈
# https://youtu.be/FtWEZw3kUho
import statsmodels.api as sm
import statsmodels.tsa.api as smt
from scipy.interpolate import UnivariateSpline

# 타입 어노테이션 모듈
from typing import *

# 시간 관련 모듈
from tqdm import tqdm
from time import strptime, sleep

# 웹크롤링 / 스크래핑 관련 모듈
import requests
import io
import zipfile
from bs4 import BeautifulSoup
from xml.etree import ElementTree as ET

# 결측값 대체 및 특징 추출
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA, SparsePCA

# 연관규칙 분석 실행 모듈
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import networkx as nx

# 한글폰트 설정
import matplotlib
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from pathlib import Path
import matplotlib.patches as mpatches

Mounted at /content/drive


**6개의 투자심리 측정변수**
* NYSE의 거래회전율
* 배당금 프리미엄
* 폐쇄형 펀드의 할인
율
* IPO의 첫 거래일 평균 수익률
* 채권 대비 주식발행 비중
* 투자심리선은 해당 종목이 과매수 또는 과매도 상태인지를 파악함으로써 투자자 심리가 낙관적인지 비관적인지를 판단하는데 사용하며, 일반적으로 75% 이상이면 해당 주식이 과매수 상태, 25% 이하이면 과매도 상태로 해석

**6개 변수를 이용하여 월별 투자 심리지수를 도출**
* ```개인투자자의 매수매도 불균형(BSI, buying and selling imbalance of individual investors)```

* ```주식형 펀드 유입액(FUND, stock fund flow ratio)```

  예금이나 채권형 펀드에서 위험한 주식형 펀드로 자금이 이동하는 현상

* ```소비자 기대지수(CEI, customer expectation index)```
  
  통계청에서 발간하는 소비자 기대지수는 경기 순환을 예측하는 지수로 본 연구에서는 자연로그를 취하여 사용(경기선행지수 중 1)

  (i) 현재생활형편, 생활형편전망, 가계수입전망, 소비지출전망, 현재경기판단, 향후경기전망 6개의 개별지수의 합산지수 산출

  (ii) 표준화하고, 6개 구성지수 표준편차의 중간값을 가중치로 곱하여 소비자 심리지수 산출

  (iii) 100보다 높으면 주관적 기대심리 과거 평균 대비 낙관적으로 해석(기준치 상회 여부 및 지수 자체의 증감 중요)

* ```주식투자 예치금(CD, customer’s deposit for stock investment)```
  
  주식투자를 위해 예치한 금액이 얼마나 변동하였는지를 측정하여 추가적으로 고려한다. 주식투자 예치금은 주식을 매수하기 위한 일시적인 예치금으로 투자자들이 주식시장에 대해 보다 낙관적인 견해를 가지고 있을수록 증가

* ```상장주식 거래회전율(TURN, turnover ratio)```

* ```주식 자금조달 비율(SR, equity shares issued ratio)```

  주식과 채권을 합계한 총 자금 조달금액에서 주식발행을 통해 조달한 자금의 비중을 사용

In [2]:
CS_DATA_M2_3_BUY1 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M2_3_BUY1.csv", index_col = False).fillna(0)
CS_DATA_M2_3_BUY2 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M2_3_BUY2.csv", index_col = False).fillna(0)
CS_DATA_M2_3_BUY3 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M2_3_BUY3.csv", index_col = False).fillna(0)
CS_DATA_M2_3_SELL1 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M2_3_SELL1.csv", index_col = False).fillna(0)
CS_DATA_M2_3_SELL2 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M2_3_SELL2.csv", index_col = False).fillna(0)
CS_DATA_M2_3_SELL3 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M2_3_SELL3.csv", index_col = False).fillna(0)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
CS_DATA_M3_1_BUY1 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M3_1_BUY1.csv", index_col = False).fillna(0)
CS_DATA_M3_1_BUY2 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M3_1_BUY2.csv", index_col = False).fillna(0)
CS_DATA_M3_1_BUY3 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M3_1_BUY3.csv", index_col = False).fillna(0)
CS_DATA_M3_1_SELL1 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M3_1_SELL1.csv", index_col = False).fillna(0)
CS_DATA_M3_1_SELL2 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M3_1_SELL2.csv", index_col = False).fillna(0)
CS_DATA_M3_1_SELL3 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M3_1_SELL3.csv", index_col = False).fillna(0)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
CS_DATA_M4_2_BUY1 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M4_2_BUY1.csv", index_col = False).fillna(0)
CS_DATA_M4_2_BUY2 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M4_2_BUY2.csv", index_col = False).fillna(0)
CS_DATA_M4_2_BUY3 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M4_2_BUY3.csv", index_col = False).fillna(0)
CS_DATA_M4_2_SELL1 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M4_2_SELL1.csv", index_col = False).fillna(0)
CS_DATA_M4_2_SELL2 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M4_2_SELL2.csv", index_col = False).fillna(0)
CS_DATA_M4_2_SELL3 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M4_2_SELL3.csv", index_col = False).fillna(0)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


```
CONSUMER_ID = pd.merge(
    CS_DATA_M2_3["고객번호"], CS_DATA_M3_1["고객번호"], how = "inner")
CONSUMER_ID = pd.merge(
    CS_DATA_M4_2["고객번호"], CONSUMER_ID, how = "inner")

for dataframe in [CS_DATA_M2_3, CS_DATA_M3_1, CS_DATA_M4_2]:
    # 150446 | 194670 | 168290
  print(len(dataframe))
len(CONSUMER_ID)
  # 91024
```

In [5]:
pd.set_option("display.max_column", None)
CS_DATA_M2_3_BUY1.info()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150446 entries, 0 to 150445
Data columns (total 57 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   월말일 기준 예수금 등 현금성 자산액             150446 non-null  float64
 1   월말일 기준 국내주식 자산 평가액               150446 non-null  float64
 2   월말일 기준 국내주식 자산 매입액               150446 non-null  float64
 3   월말일 기준 해외주식 자산 평가액               150446 non-null  float64
 4   월말일 기준 해외주식 자산 매입액               150446 non-null  float64
 5   월말일 기준 국내주식 보유종목수                150446 non-null  float64
 6   월말일 기준 해외주식 보유종목수                150446 non-null  float64
 7   특정기간동안의 주식 매수체결횟수                150446 non-null  float64
 8   특정기간동안의 주식 매도체결횟수                150446 non-null  float64
 9   특정기간동안의 주식 매수종목수                 150446 non-null  float64
 10  특정기간동안의 주식 매도종목수                 150446 non-null  float64
 11  특정기간동안의 주식 매수액 합계                150446 non-null  float64
 12  특정

* 투자심리지수 파생컬럼

0) **10세 기준 연령대, 관리사원 존재여부**

1) **전환율(CTR)**

    매수전환율 = 매수체결횟수 / 접속일수
    
    매도전환율 = 매도체결횟수 / 접속일수

* CONN_DYS < MTS_DYS + HTS_DYS
* 특정기간동안의 HTS 또는 MTS 접속일수

2) **최고자산 대비 등락률**

    {(월말일 기준 예수금 등 현금성 자산액 + 월말일 기준 국내주식 자산 평가액 + 월말일 기준 해외주식 자산 평가액) - (과거 최고자산액)} / (과거 최고자산액) * 100

    특이 시점 - 과거 최고자산액 기록 연월

3) **거래회전율 = 1월부터 3월까지 총 거래량 / 특정기간동안의 거래량**
  
    주식매수대금회전율 = 3개월간 주식 매수액 합계 / 특정기간동안의 주식 매수액 합계

    주식매도대금회전율 = 3개월간 주식 매도액 합계 / 특정기간동안의 주식 매도액 합계
    
    주식매수량회전율 = 3개월간 주식 매수종목수 합계 / 특정기간동안의 주식 매수종목수 합계
    
    주식매도량회전율 = 3개월간 주식 매도종목수 합계 / 특정기간동안의 주식 매도종목수 합계

4) **주식수익률**

    국내주식수익률 = 월말일 기준 국내주식 자산 평가액 / 월말일 기준 국내주식 자산 매입액
    해외주식수익률 = 월말일 기준 국내주식 자산 평가액 / 월말일 기준 국내주식 자산 매입액


5) **매수매도 불균형(IBSI)**

    매수매도불균형 = 특정기간동안의 주식 매수액 합계 / 특정기간동안의 주식 매도액 합계

6) **대체입출고비중[타 증권사 주식이체]  >= 50%**
    
    유가증권대체입고[진입율] = (특정기간동안의 주식 입고액 합계 + 특정기간동안의 입금액 합계) / (월말일 기준 예수금 등 현금성 자산액 + 월말일 기준 국내주식 자산 평가액)

    유가증권대체출고[이탈률] = (특정기간동안의 주식 출고액 합계 + 특정기간동안의 출금액 합계) / (월말일 기준 예수금 등 현금성 자산액 + 월말일 기준 국내주식 자산 평가액)
    


In [ ]:
CS_DATA_M2_3["거래회전율"]
CS_DATA_M2_3["주식수익률"]
CS_DATA_M2_3["거래회전율"]



